# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
#csv_file = os.path.join('..', 'WeatherPy', 'city_data.csv')

#city_df = pd.read_csv(csv_file)
#city_df



,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed


In [2]:
#import city_data.csv from part 1
city_data_df = pd.read_csv('city_data.csv')
city_data_df.count()

Unnamed: 0     545
City           545
Latitude       545
Longitude      545
Temperature    545
Humidity       545
Cloudiness     545
Wind Speed     545
Country        541
Date           545
dtype: int64

In [3]:
city_data_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Colombia,10.9878,-74.9547,87.66,74,99,20.36,CO,1620848237
1,1,Yumen,40.2833,97.2000,58.86,25,62,10.47,CN,1620848237
2,2,Anchorage,61.2181,-149.9003,48.16,53,1,0.00,US,1620848168
3,3,Nikolskoye,59.7035,30.7861,64.02,72,1,3.85,RU,1620848238
4,4,Cabo San Lucas,22.8909,-109.9124,90.12,37,20,8.05,MX,1620848211


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = city_data_df[['Latitude','Longitude']]
humidity = city_data_df['Humidity']

#create a gmap figure
fig = gmaps.figure()

#add the heatmap layer
fig.add_layer(gmaps.heatmap_layer(locations,weights=humidity,dissipating=False, max_intensity=100, point_radius=2))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

best_vaca_df = city_data_df.loc[(city_data_df['Temperature'] < 80) & (city_data_df['Temperature']>=70)\
                         & (city_data_df['Wind Speed'] < 10)\
                         & (city_data_df['Cloudiness'] == 0)] 

#store as a new variable
hotel_df = best_vaca_df.copy()

hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
159,159,Le Port,-20.9373,55.2919,76.78,75,0,2.66,RE,1620848289
343,343,Karratha,-20.7377,116.8463,76.78,42,0,5.55,AU,1620848351
516,516,Saint-Pierre,-21.3393,55.4781,72.84,83,0,6.91,RE,1620848411


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    
#iterate through the rows of the df
for index, row in hotel_df.iterrows():
    
    coordinates = f"{row['Latitude']},{row['Longitude']}"
    params = {"location": coordinates, "radius": 5000, "type": 'lodging', "key": g_key}
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except(KeyError, IndexError):
        print('Missing Field. Skipping...')

Missing Field. Skipping...
Missing Field. Skipping...
Missing Field. Skipping...


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

KeyError: 'Hotel Name'

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))